In [1]:
# 卷积神经网络
# MLP 参数量过大 不合适
# 不如记忆所有图片

In [2]:
# 两个原则
# 平移不变性
# 局部性
# 图片处理 启发CNN设计
# CNN 本质上是 MLP 的简化
# 应用上述两个原则(平移不变性和局部性)

In [4]:
# 卷积层
# 卷积实际上是交叉相关
# 没有严格遵循卷积的定义
# 1D,2D,3D 交叉相关(卷积)
# 卷积层:将输入与核矩阵进行交叉相关,加上偏移
# 可学习:核矩阵和偏移
# 超参数:核矩阵的大小

In [5]:
# 图像卷积
import torch
from torch import nn 
from d2l import torch as d2l

In [20]:
from tkinter import W


def corr2d(X,K):
    """计算二维互相关"""
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [21]:
X=torch.tensor(
    [[0.,1.,2.],
     [3.,4.,5.],
     [6.,7.,8.]]
)
K=torch.tensor(
    [[0.,1.],
     [2.,3.]]
)
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [22]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))
        
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [23]:
X=torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [25]:
K=torch.tensor([[1.0,-1.0]])

In [28]:
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [27]:
corr2d(X.t(),K)
# 只能检测垂直边缘 不能检测水平边缘

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [29]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))

X,Y

(tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.]]]]),
 tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]]))

In [31]:
for i in range(20):
    Y_hat=conv2d(X)
    l=(Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:]-=3e-2*conv2d.weight.grad
    if (i+1)%2==0:
        print(f"epoch {i+1}, loss {l.sum():.3f}")

epoch 2, loss 0.081
epoch 4, loss 0.033
epoch 6, loss 0.013
epoch 8, loss 0.005
epoch 10, loss 0.002
epoch 12, loss 0.001
epoch 14, loss 0.000
epoch 16, loss 0.000
epoch 18, loss 0.000
epoch 20, loss 0.000


In [32]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9995, -1.0005]])

In [33]:
# 同样表达能力的深度卷积的参数比浅层更少 而且非线性性更好

In [34]:
# 卷积层的填充和步幅
# 填充 Padding
# 越卷越小
# 在输出周围添加额外行列
# 步幅 stride
# 填充和步幅是卷积层的超参数


In [35]:
def comp_conv2d(conv2d,X):
    X=X.reshape((1,1)+X.shape)
    Y=conv2d(X)
    return Y.reshape(Y.shape[2:])


In [36]:
# padding 指左右都填充了1行
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1)
X=torch.rand(size=(8,8))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [37]:
# padding 不对称
conv2d=nn.Conv2d(1,1,kernel_size=(5,3),padding=(2,1))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [38]:
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [40]:
conv2d=nn.Conv2d(1,1,kernel_size=(3,5),padding=(0,1),stride=(3,4))
comp_conv2d(conv2d,X).shape

torch.Size([2, 2])

In [42]:
# 一般使用对称的情况
# 除非原始数据就shape偏差
# 填充通常选2p=核-1
# 保证size不变
# 步幅=1最好 计算量过大 增加步幅=2
# 均匀的插步幅=2的层
# 奇数的核大小便于对称填充
# 步幅和填充是架构的一部分
# 除非数据特别奇怪，不然使用经典架构
# 网络对训练的影响其实大家也不太知道
# 自己设计 --数据集特殊 --硬件特殊
# 机器学习 --压缩算法 --丢失信息
# 计算机的像素信息-->人能理解的语义信息
# 小的卷积核能减少计算量 但是效果相同
# 简单的网络更容易被应用


In [43]:
# 多个输入和输出通道

In [44]:
# 1*1 卷积层
# 不识别空间信息,只是融合通道(加权和)
# 每个输入空间的共享参数的全连接层
# channel维的全连接层(共享参数)

In [45]:
# 输出通道数是卷积层的超参数
# 输入通道 独立二维卷积核
# 输出通道 独立三维卷积核
# Filter 四维张量

In [46]:
# 多输入和多输出通道
def corr2d_multi_in(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [55]:
X=torch.tensor([np.arange(9).reshape(3,3),np.arange(9).reshape(3,3)+1])
X

tensor([[[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]],

        [[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]], dtype=torch.int32)

In [64]:
import numpy as np


K=torch.tensor([np.arange(4).reshape(2,2),np.arange(4).reshape(2,2)+1])
K

tensor([[[0, 1],
         [2, 3]],

        [[1, 2],
         [3, 4]]], dtype=torch.int32)

In [56]:
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [60]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

In [65]:
K=torch.stack((K,K+1,K+2),0)
K.shape
# 3个输出，2个输入，高2，宽2

torch.Size([3, 2, 2, 2])

In [66]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [67]:
# 1*1 卷积
from inspect import CO_ASYNC_GENERATOR


def corr2d_multi_in_out_1_1(X,K):
    c_i,h,w=X.shape
    c_o=K.shape[0]
    X=X.reshape((c_i,h*w))
    K=K.reshape((c_o,c_i))
    # 转化成类似全连接
    Y=torch.matmul(K,X)
    return Y.reshape((c_o,h,w))

In [68]:
X=torch.normal(0,1,(3,3,3))
K=torch.normal(0,1,(2,3,1,1))

Y1=corr2d_multi_in_out_1_1(X,K)
Y2=corr2d_multi_in_out(X,K)

float(torch.abs(Y1-Y2).sum())

0.0

In [70]:
# pytorch 调用
# 见前面

In [71]:
# 池化层
# 积对位置敏感
# 需要一定程度的平移不变性
# 1像素移位导致0输出
# 照明，物体位置，比例，外观
# 二维最大池化
# 允许输入发生小小的偏移
# 相当于一种数据的模糊处理
# 有填充和步幅
# 没有可学习的参数
# 输出通道数=输入通道数

In [73]:
def pool2d(X,pool_size,mode="max"):
    p_h,p_w=pool_size
    Y=torch.zeros((X.shape[0]-p_h+1,X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode=="max":
                Y[i,j]=X[i:i+p_h,j:j+p_w].max()
            elif mode=="avg":
                Y[i,j]=X[i:i+p_h,j:j+p_w].mean()
    return Y

In [76]:
X=torch.tensor(range(9),dtype=torch.float32).reshape(3,3)
pool2d(X,(2,2))

tensor([[4., 5.],
        [7., 8.]])

In [77]:
pool2d(X,(2,2),"avg")

tensor([[2., 3.],
        [5., 6.]])

In [78]:
X=torch.arange(16,dtype=torch.float32).reshape((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [81]:
pool2d=nn.MaxPool2d(3,stride=1)
pool2d(X)

tensor([[[[10., 11.],
          [14., 15.]]]])

In [80]:
pool2d=nn.MaxPool2d(2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [82]:
pool2d = nn.MaxPool2d(1)
pool2d(X)


tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [83]:
# 深度学习框架中默认步幅与池化窗口大小相同

In [85]:
# 可设定任意大小
pool2d=nn.MaxPool2d(3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [86]:
pool2d=nn.MaxPool2d((2,3),padding=(1,1),stride=(2,3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [87]:
X=torch.cat((X,X+1),1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [88]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

In [ ]:
# 少调用python的函数
# 矩阵不用管，会自己释放内存
# List.append 性能比 tensor.cat 好很多
# 数据做增强 池化的效果不大 数据本身就有相当扰动
# 数据保证不会过拟合 淡化池化的作用